# PyTorch CNN implementation for handwritten digit recognition MNIST

**Author:** Miguel Robledo Kusz

**Description:** This project is mainly for learning purposes. It starts with a basic implementation of a CNN in PyTorch to recognize handwritten digits from the MNIST dataset. Later, I plan to extend it by testing the model on digits written by myself, creating a small custom dataset. This way, I can evaluate the model’s performance in different scenarios.

**Use of LLMs for assistance:** I acknowledge the major role that LLMs play today in development, both for generating code and improving documentation. I will use them as a support tool in this project, for commands, PyTorch syntax, and general guidance, but always applying critical thinking and making sure I understand what I’m doing. The ultimate goal is to learn to carry out these tasks independently in the future. *Miguel Robledo Kusz, October 3rd, 2025, 00:41.*

## Importing libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'